<a href="https://colab.research.google.com/github/wolfzxcv/ml-examples/blob/master/TSP_using_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Key Components of a Genetic Algorithm**

* Selection: Choosing individuals from the population to create offspring.
* Crossover: Combining parts of two individuals to create a new individual.
* Mutation: Introducing small random changes to individuals to maintain genetic diversity.

In [22]:
from random import randint, choice

# A very high value to represent no direct path between cities
INT_MAX = 2147483647

# Number of cities in TSP
V = 5

# Names of the 5 cities
GENES = "ABCDE"

# Initial population size (potential solutions) for the algorithm
POP_SIZE = 10

# Max number of generations
max_gen = 20

# A list to store the population of individuals
population = []

In [23]:
# individual solution (a possible path)
class Individual:
    def __init__(self, gnome, fitness):
        # A string representing the path (e.g., "012340")
        self.gnome = gnome
        # The total distance of the path
        self.fitness = fitness


# Generate a random number between start and end-1
def rand_num(start, end):
    return randint(start, end - 1)  # Adjust for Python's zero-based indexing


# Generates a valid initial path (gnome) starting and ending at the first city
def create_gnome():
    gnome = str(choice(range(V)))
    while len(gnome) < V:
        temp = rand_num(0, V)
        if str(temp) not in gnome:
            gnome += str(temp)
    return gnome


# Perform crossover between two parents to create offspring
def crossover(parent1, parent2):
    child_p1 = ""
    child_p2 = ""

    geneA = int(rand_num(1, V))
    geneB = int(rand_num(1, V))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        child_p1 += parent1.gnome[i]

    child_p2 = [item for item in parent2.gnome if item not in child_p1]

    child = child_p1 + ''.join(child_p2)

    return child


# Mutates a gnome by swapping two random cities in the path
def mutated_gene(gnome):
    gnome = list(gnome)
    while True:
        r = rand_num(1, V)
        r1 = rand_num(1, V)
        if r1 != r:
            temp = gnome[r]
            gnome[r] = gnome[r1]
            gnome[r1] = temp
            break
    return ''.join(gnome)


# Calculates the fitness of a gnome by summing the distances between continuous cities in the path
def calculate_fitness(gnome):
    # distance_matrix represents the distances between cities
    # The fitness value is the total distance traveled
    # Summing up the distances between continuous cities along the path
    distance_matrix = [
        [0, 2, INT_MAX, 12, 5],
        [2, 0, 4, 8, INT_MAX],
        [INT_MAX, 4, 0, 3, 3],
        [12, 8, 3, 0, 10],
        [5, INT_MAX, 3, 10, 0],
    ]
    fitness = 0
    for i in range(V - 1):
        fitness += distance_matrix[int(gnome[i])][int(gnome[i + 1])]
        # Add the distance from the last city back to the starting city
        fitness += distance_matrix[int(gnome[-1])][int(gnome[0])]
    return fitness

In [24]:
# Main function for TSP problem.
def tsp_util():
    global population, max_gen  # Declare population and max_gen as global

    # Current generation number
    gen = 1

    print("Initial population: \nGNOME FITNESS VALUE")
    # Populating the GNOME pool.
    for i in range(POP_SIZE):
        gnome = create_gnome()
        fitness = calculate_fitness(gnome)
        print(gnome, fitness)
        population.append(Individual(gnome, fitness))

    # Iteration to perform
    while gen <= max_gen:
      population.sort(key=lambda x: x.fitness)
      new_population = []

      # Elitism: Carry over the best individuals to the next generation
      elites_count = int(POP_SIZE * 0.5)  # Keep top 50% of the population as elites
      elites = population[:elites_count]
      new_population.extend(elites)

      # Selection, Crossover, and Mutation
      for _ in range(POP_SIZE - elites_count):
          parent1 = choice(population)
          parent2 = choice(population)
          child_gnome = crossover(parent1, parent2)
          mutated_gnome = mutated_gene(child_gnome)
          new_fitness = calculate_fitness(mutated_gnome)
          new_population.append(Individual(mutated_gnome, new_fitness))

      population = new_population

      print(f"\nGeneration {gen}")
      print("GNOME FITNESS VALUE")

      for ind in population:
          print(f"{ind.gnome} {ind.fitness}")

      gen += 1

    # Handle result
    # Find the minimum fitness value
    min_fitness = min(individual.fitness for individual in population)

    # Find all individuals with the minimum fitness value
    best_individuals = [individual for individual in population if individual.fitness == min_fitness]

    # Print the smallest fitness value (shortest path)
    print(f"\nShortest path (smallest fitness value in the last iteration): {min_fitness}")

    # Decode the gnome (visiting order) and print the unique solutions
    unique_results = set()

    for best_individual in best_individuals:
        best_gnome = best_individual.gnome
        result = "".join([GENES[int(city)] for city in best_gnome][::-1])  # List comprehension with reverse
        visiting_order = f"{result}{result[0]}"  # Add starting city at the end
        if visiting_order not in unique_results:
            unique_results.add(visiting_order)
            print(f"Visiting order (city names): {visiting_order}")


if __name__ == "__main__":
    tsp_util()

Initial population: 
GNOME FITNESS VALUE
02341 4294967315
40231 10737418251
14320 4294967315
40132 30
01342 8589934611
04231 27
31204 2147483704
41320 4294967325
24103 2147483676
31420 4294967353

Generation 1
GNOME FITNESS VALUE
04231 27
40132 30
24103 2147483676
31204 2147483704
02341 4294967315
40132 30
34201 2147483694
43102 2147483679
24031 44
31024 2147483700

Generation 2
GNOME FITNESS VALUE
04231 27
40132 30
40132 30
24031 44
24103 2147483676
42031 10737418258
10423 45
32041 4294967334
20341 4294967332
24310 8589934611

Generation 3
GNOME FITNESS VALUE
04231 27
40132 30
40132 30
24031 44
10423 45
01423 2147483703
41032 2147483676
03241 2147483673
40312 41
02413 4294967353

Generation 4
GNOME FITNESS VALUE
04231 27
40132 30
40132 30
40312 41
24031 44
14023 4294967334
04123 2147483707
04123 2147483707
01234 39
40321 8589934612

Generation 5
GNOME FITNESS VALUE
04231 27
40132 30
40132 30
01234 39
40312 41
04213 68
21403 2147483680
31042 30
03214 2147483686
42310 36

Generation 6
G